First Casino game will be BlackJack. 

Defining the class and the attributes that will be necessary to a smooth game function.

In [1]:
import random
from tkinter import Tk, Label
from PIL import Image, ImageTk

class BlackJack:
    def __init__(self, num_players, table_minimum, player_balance):
        self.window = Tk()
        self.window.title('Blackjack')

        # Eventually, hopefully loading card images.  For now Place Holder Comment

        # Game operation variables
        self.num_players = num_players
        self.num_deck = 8
        self.player_hands = [[] for _ in range(num_players)]
        self.dealer_hand = []
        self.table_minimum = table_minimum
        self.player_balance = player_balance
        self.pot = 0
        self.dealer_shows = None
        self.player_status = [True] * num_players
        self.deal_occurred = False
        self.percent_filled = 1.0  # Starting with a full shoe
        self.player_turn_order = list(range(0, num_players))
        self.player_status = [False] * num_players # Tracks if a player's turn is complete, False means they still need to take game actions


# Simulating a casino style shuffle and game of blackjack.
    def fill_shoe(self):  
        # Defining my deck
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        
        # Creating a single deck
        deck = [(rank, suit) for suit in suits for rank in ranks]
        
        # Creating the shoe and shuffling it
        shoe = deck * self.num_deck
        random.shuffle(shoe)
        
        return shoe


# Handling the intial betting of a blackjack game, that occurs prior to cards being dealt
    def initial_bet(self): 
        for i in range(self.num_players):
            while True:
                bet = int(input(f'Player {i+1}, place your bet (min bet: {self.table_minimum}): '))
                if bet >= self.table_minimum:
                    self.pot += bet
                    self.player_balance -= bet
                    break
                else:
                    print(f'Bet must be at least {self.table_minimum}. Please try again.')


# Simulating a casino style round robin deal where players see each other's cards and only 1 of the dealer's cards.
    def deal(self): 
        # Deal one card to each player, and one to the dealer (these are all visible to players)
        for player_hand in self.player_hands:
            player_hand.append(self.shoe.pop())
        self.dealer_hand.append(self.shoe.pop())
        self.dealer_shows = self.dealer_hand[0]  # The dealer's visible card

        # Deal a second card to each player and one to dealer (dealer's second card remains hidden)
        for player_hand in self.player_hands:
            player_hand.append(self.shoe.pop())
        self.dealer_hand.append(self.shoe.pop()) # Second card remains hidden for dealer

        self.deal_occurred = True

        # Tracking the percentage of cards remaining in the shoe
        max_num_cards = 52 * self.num_deck
        remaining_cards = len(self.shoe)
        self.percent_filled = remaining_cards / max_num_cards

        if self.percent_filled < 0.2:
            print("Shoe is low on cards, it will be refilled after this round.")


    # Refilling the shoe after enough cards have been dealt.
    def refill_shoe(self):
        self.shoe = self.fill_shoe()
        self.percent_filled = 1.0

    def manage_all_player_actions(self):
        for player_index in self.player_turn_order:
            if not self.player_status[player_index]:
                print(f"Player {player_index + 1}'s turn:")
                self.manage_player_actions(player_index)


    # This function will maintain individual player turn actions.  Player 1 completes their actions before 
    # control is passed to Player 2 and so on.
    def manage_player_actions(self, player_index):
        # Clear any existing widgets in the window
        for widget in self.window.winfo_children():
            widget.destroy()
        
        # Display the current player
        turn_label = tk.Label(self.window, text = f"Player {player_index + 1}'s turn")
        turn_label.pack()
        
        # Create all the possible buttons a player can interact with
        hit_button = tk.Button(self.window, text = 'Hit', command = lambda: self.hit(player_index))
        stand_button = tk.Button(self.window, text = 'Stand', command = lambda: self.stand(player_index))
        split_button = tk.Button(self.window, text = 'Spilt', command = lambda: self.spilt(player_index))
        double_down_button = tk.Button(self.window, text = 'Double Down', command = lambda: self.double_down(player_index))
        insure_button = tk.Button(self.window, text = 'Insure', command = lambda: self.insure(player_index))
        
        # Check the player's hand for valid turn actions
        valid_actions = self.get_valid_actions(player_index)

        # Check if the player has busted
        if 'bust' in valid_actions:
            print(f'Player {player_index + 1} busts!')
            self.player_status[player_index] = True
            return 

        # Display the appropriate buttons for the player to choose from
        if "hit" in valid_actions:
            hit_button.pack()
        if "stand" in valid_actions:
            stand_button.pack()
        if "split" in valid_actions:
            split_button.pack()
        if "double_down" in valid_actions:
            double_down_button.pack()
        if "insure" in valid_actions:
            insure_button.pack()
        
        # Update the window to reflect changes
        self.window.update_idletasks()
        self.window.update()


    # Check a player's hand for appropriate or valid actions
    def valid_actions(self, player_index):
        valid_actions = []

        # Get the player's current hand
        hand = self.player_hands[player_index]

        # First check if player has busted
        if self.calculate_hand_value(hand) > 21:
            valid_actions.append('bust')
            return(valid_actions)

        # Check if insurance is available
        if self.dealer_show_card[0] == 'Ace':
            valid_actions.append('insure')
        
        # Check for two cards in hand (determines if double_down or splitting is available)
        if len(hand) == 2:
            valid_actions.extend(['hit', 'stand', 'double_down'])
            if self.can_split(hand):
                valid_actions.append('split')
        else:
            valid_actions.extend(['hit', 'stand'])

        return valid_actions


    def can_split(self, hand):
        return hand[0][0] == hand[1][0]


    def calculate_hand_value(self, hand):
        # Define the scoring dictionary to handle card values.
        # NOTE the value of an Ace is set to 11.  This is to avoid a keyerror when attempting to calculate the score of a hand that includes an 
        # Ace while indexing this dictionary.
        scoring_dict = {'Ace': 11, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'Jack': 10, 'Queen': 10, 'King': 10}
        
        num_of_aces = 0
        score = 0

        for card in hand:
            rank = card[0]
            score += scoring_dict[rank]
            if rank == 'Ace':
                num_of_aces += 1
        
        while score > 21 and num_of_aces > 0:
            score -= 10
            num_of_aces -= 1
        
        return score


    # Functions for player actions
    def hit(self, player_index):
        # Player chooses to take another card
        self.player_hands[player_index].append(self.shoe.pop())

        hand_value = self.calculate_hand_value(self.player_hands[player_index])
        print(f"Player {player_index + 1}'s hand: {self.player_hands[player_index]}")

        # Continue Player's turn within the manage_player_actions function
        self.manage_player_actions(player_index)


    def stand(self, player_index):
        # Player choose to stop taking cards and stay at their score
        self.player_status[player_index] = True
        hand = self.player_hands[player_index]
        score = self.calculate_hand_value(hand)
        print(f"Player {player_index + 1} stands at {score}")

In [ ]:


def split(self):
    # If player's first two cards are of the same rank, player chooses to split cards into two hands

def double_down(self):
    # Player choose to double the initial bet and take exactly one more card.

    # Logic here is similar to hit and then stand but should be unique here and not call those two functions since they pass control
    # back to other functions.

def insure(self):
    # Offered to player when the dealer's visible card is an Ace, allowing a side bet that the dealer has Blackjack

def place_bets(self):
    self.pot = 0
    for player in range(1, self.num_players + 1):
        if self.player_status[player] == True and self.deal_occurred == False:
            bet = int(input(f'Player {player + 1}, place your bet: '))
            self.pot += bet
        else:
            pass
